<a href="https://colab.research.google.com/github/ancestor9/2025_Fall_AI-Model-Operations-MLOps/blob/main/week07/database_python/sqlite3_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import os

# 데이터베이스 파일 이름
DB_NAME = "test_sqlite3.db"

def init_db():
    """데이터베이스 파일을 생성하고 연결합니다."""
    # 파일이 이미 존재하면 삭제 (이전 실행 결과로 인한 문제 방지)
    if os.path.exists(DB_NAME):
        os.remove(DB_NAME)
        print(f"이전 데이터베이스 파일 '{DB_NAME}' 삭제 완료.")

    # 1. 데이터베이스 연결 (파일이 없으면 생성)
    conn = sqlite3.connect(DB_NAME)
    # Cursor 객체 생성 (SQL 명령을 실행하는 객체)
    cursor = conn.cursor()

    # 2. 테이블 생성 (CREATE)
    print("\n--- [CREATE] 테이블 생성 ---")
    try:
        cursor.execute("""
            CREATE TABLE users (
                id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                fullname TEXT
            )
        """)
        conn.commit() # 변경사항 확정 (커밋)
        print("테이블 'users' 생성 완료.")
    except sqlite3.OperationalError as e:
        print(f"테이블 생성 오류: {e}")

    return conn

def crud_operations(conn):
    """주요 CRUD 작업을 수행합니다."""
    cursor = conn.cursor()

    # C (Create) - 데이터 삽입
    print("\n--- [C]reate: 새 사용자 추가 ---")
    users_data = [
        ('spongebob', 'Spongebob Squarepants'),
        ('patrick', 'Patrick Star'),
        ('squidward', 'Squidward Tentacles')
    ]
    # 여러 행을 한 번에 삽입 (튜플 리스트 사용)
    cursor.executemany("INSERT INTO users (name, fullname) VALUES (?, ?)", users_data)
    conn.commit()
    print(f"{len(users_data)}개의 사용자 데이터 삽입 완료.")

    # R (Read) - 데이터 읽기 (전체 조회)
    print("\n--- [R]ead: 모든 사용자 조회 ---")
    cursor.execute("SELECT id, name, fullname FROM users")
    rows = cursor.fetchall() # 모든 결과를 리스트 형태로 가져오기

    for row in rows:
        print(f"ID: {row[0]}, Name: {row[1]}, FullName: {row[2]}")

    # R (Read) - 특정 조건 조회
    print("\n--- [R]ead: 특정 사용자 (name='patrick') 조회 ---")
    cursor.execute("SELECT * FROM users WHERE name = ?", ('patrick',))
    patrick_row = cursor.fetchone() # 하나의 결과만 가져오기
    if patrick_row:
        print(f"조회된 사용자: {patrick_row}")

    # U (Update) - 데이터 수정
    print("\n--- [U]pdate: 사용자 이름 수정 (Spongebob의 fullname) ---")
    new_fullname = 'S. Squarepants'
    cursor.execute("UPDATE users SET fullname = ? WHERE name = ?", (new_fullname, 'spongebob'))
    conn.commit()
    print(f"스폰지밥의 FullName이 '{new_fullname}'로 수정 완료.")

    # D (Delete) - 데이터 삭제
    print("\n--- [D]elete: 사용자 (name='squidward') 삭제 ---")
    cursor.execute("DELETE FROM users WHERE name = ?", ('squidward',))
    conn.commit()
    print("Squidward 사용자 삭제 완료.")

    # R (Read) - 최종 확인
    print("\n--- [R]ead: 최종 데이터 확인 ---")
    cursor.execute("SELECT * FROM users")
    final_rows = cursor.fetchall()

    if final_rows:
        for row in final_rows:
            print(f"최종 데이터: {row}")
    else:
        print("남아있는 데이터가 없습니다.")


# --- 메인 실행 ---
if __name__ == "__main__":

    # 데이터베이스 초기화 및 연결
    conn = init_db()

    try:
        # CRUD 작업 실행
        crud_operations(conn)

    finally:
        # 3. 연결 종료 (매우 중요)
        conn.close()
        print(f"\n데이터베이스 '{DB_NAME}' 연결 종료.")


--- [CREATE] 테이블 생성 ---
테이블 'users' 생성 완료.

--- [C]reate: 새 사용자 추가 ---
3개의 사용자 데이터 삽입 완료.

--- [R]ead: 모든 사용자 조회 ---
ID: 1, Name: spongebob, FullName: Spongebob Squarepants
ID: 2, Name: patrick, FullName: Patrick Star
ID: 3, Name: squidward, FullName: Squidward Tentacles

--- [R]ead: 특정 사용자 (name='patrick') 조회 ---
조회된 사용자: (2, 'patrick', 'Patrick Star')

--- [U]pdate: 사용자 이름 수정 (Spongebob의 fullname) ---
스폰지밥의 FullName이 'S. Squarepants'로 수정 완료.

--- [D]elete: 사용자 (name='squidward') 삭제 ---
Squidward 사용자 삭제 완료.

--- [R]ead: 최종 데이터 확인 ---
최종 데이터: (1, 'spongebob', 'S. Squarepants')
최종 데이터: (2, 'patrick', 'Patrick Star')

데이터베이스 'test_sqlite3.db' 연결 종료.


###